### 8. Поиск аномалий (система алертов)
##### Задача: напишите систему алертов для нашего приложения
Система должна с периодичностьç каждые 15 минут проверять ключевые метрики, такие как активные пользователи в ленте / мессенджере, просмотры, лайки, CTR, количество отправленных сообщений. 
1. Изучите поведение метрик и подберите наиболее подходящий метод для детектирования аномалий. 
Проверяю отклонение значения метрики в текущую 15-минутку от значения в такую же 15-минутку день назад. 
2. В случае обнаружения аномального значения, в чат должен отправиться алерт - сообщение со следующей информацией: метрика, ее значение, величина отклонения. В сообщение можно добавить дополнительную информацию, которая поможет при исследовании причин возникновения аномалии, это может быть, например,  график, ссылки на дашборд/чарт в BI системе. 
3. Автоматизируйте систему алертов с помощью Airflow

#### Сравниваню интересующее значение ключевой метрики за 15-минутку со значением в это же время сутки назад.

In [ ]:
import numpy as np # для нахождения даты 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import telegram
import pandahouse as ph
from datetime import date, datetime, timedelta
import io # сохраняю картинки в буфере

#скрыть токен и chat_id, когда выложу в гите
import sys
import os

import math

from airflow.decorators import dag, task
#from airflow.operators.python import get_current_contex


#для подключения к clickhouse
connection = {
              'host':'https://clickhouse.lab.karpov.courses',
              'database':'simulator_20221020',
              'user':'student', 
              'password':'dpo_python_2020'
             }


# Дефолтные параметры, которые прокидываются в таски
default_args = {
                'owner': 'aksakal',
                'depends_on_past': False,
                'retries': 2,
                'retry_delay': timedelta(minutes=5),
    
                'email': 'bailukova.bk@gmail.com', # Почта для уведомлений
                'email_on_failure': 'bailukova.bk@gmail.com', # Почта для уведомлений при ошибке
                'email_on_retry': 'bailukova.bk@gmail.com', # Почта для уведомлений при перезапуске
    
                'start_date': datetime(2022, 11, 9), # старт дата, с которой собираем данные9/11/2022
                }

# Интервал запуска DAG
schedule_interval = '*/15 * * * *'


#функция поиска аномалиий c помощью алгоритма  аномалий в данных (межквартильный размах)
def check_anomaly(data, metric, threshold=0.5):
    # функция check_anomaly предлагает алгоритм проверки значения на аномальность посредством
    # сравнения интересующего значения со значением в это же время сутки назад
    # при желании алгоритм внутри этой функции можно изменить
    current_ts = df['ts'].max()  # достаем максимальную 15-минутку из датафрейма - ту, которую будем проверять на аномальность
    
    if metric == 'users_message':
        day_ago_ts = current_ts - pd.DateOffset(days=3)  # достаем такую же 15-минутку сутки назад
    else:
        day_ago_ts = current_ts - pd.DateOffset(days=1)  # достаем такую же 15-минутку сутки назад


    current_value = df[df['ts'] == current_ts][metric].iloc[0] # достаем из датафрейма значение метрики в максимальную 15-минутку
    day_ago_value = df[df['ts'] == day_ago_ts][metric].iloc[0] # достаем из датафрейма значение метрики в такую же 15-минутку сутки назад

    # вычисляем отклонение
    if current_value <= day_ago_value:
        diff = abs(current_value / day_ago_value - 1)
    else:
        diff = abs(day_ago_value / current_value - 1)

    # проверяем больше ли отклонение метрики заданного порога threshold
    # если отклонение больше, то вернем 1, в противном случае 0
    if diff > threshold:
        is_alert = 1
    else:
        is_alert = 0 

    return is_alert, current_value, diff




@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def aksakal_alert_less():

    today = np.datetime64('today')

    # считаю средние показатели метрик за 7 дней до сегодня + ставлю дату за вчера
    @task
    def extract_feed_7day():
        query = ''' SELECT 
                              hm 
                            , avg(users_feed) as users_feed
                            , avg(views) as views
                            , avg(likes) as likes
                            , avg(ctr) as ctr
                            , date
                    FROM
                        (SELECT
                              formatDateTime(toStartOfFifteenMinutes(time), '%R') as hm
                            , uniqExact(user_id) as users_feed
                            , countIf(user_id, action = 'view') as views
                            , countIf(user_id, action = 'like') as likes
                            , likes/views as ctr
                            , today()-1 as date
                        FROM 
                            {db}.feed_actions
                        WHERE 
                            time >=  today() - 7 and time < today()

                        GROUP BY  date, hm
                        ORDER BY date, hm)

                    GROUP BY  date, hm
                    ORDER BY date, hm
                    '''
        df_cube_feed_7day = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_7day


    # считаю метрики только за сегодня
    @task
    def extract_feed_today():
        query = ''' SELECT
                          toStartOfFifteenMinutes(time) as ts
                        , toDate(time) as date
                        , formatDateTime(ts, '%R') as hm
                        , uniqExact(user_id) as users_feed
                        , countIf(user_id, action = 'view') as views
                        , countIf(user_id, action = 'like') as likes
                        , likes/views as ctr

                    FROM 
                        {db}.feed_actions
                    WHERE 
                        time >= today() and time < toStartOfFifteenMinutes(now())

                    GROUP BY date, hm, ts
                    ORDER BY date, hm 
                    '''
        df_cube_feed_today = ph.read_clickhouse(query, connection=connection)
        return df_cube_feed_today


    # WHERE time >=  today() and time < toStartOfFifteenMinutes(now()) - беру период за вчера до сегодня без текущих 15 минут,
    # тк текущая 15-минутка может быть неполная и тогда получу алерт, что данные упали, хотя это не так
    # toStartOfFifteenMinutes(now()) - округляет текущее время до начала текущей 15 минутки 
    # formatDateTime(time, '%R') as hm  - округленное до 15 минуток время выводит в формате часы-минуты 
    # добавила колонки date, hm - для удобства построения графиков 


    # считаю метрики за 3 дня до сегодня + сегодня для users_message, тк там есть закономерность каждые 3 дня
    @task
    def extract_users_message():
        query = ''' SELECT 
                            ts, date, hm, users_message
                    FROM

                        (SELECT
                              toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as users_message

                        FROM 
                          {db}.message_actions
                        WHERE 
                           time >= today() and time < toStartOfFifteenMinutes(now())

                        GROUP BY date, hm, ts
                        ORDER BY date)

                        UNION ALL               

                       (SELECT
                              toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as users_message

                        FROM 
                          {db}.message_actions
                        WHERE 
                            time >= today()-3 and time < today()-2

                        GROUP BY date, hm, ts
                        ORDER BY date)
                    '''
        df_cube_users_message = ph.read_clickhouse(query, connection=connection)
        return df_cube_users_message


    # считаю средние показатели метрик за 10 дней до сегодня + ставлю дату за вчера
    # буду смотреть аномалии по алгоритму для метрик feed 
    @task
    def extract_sent_message():
        query = ''' 
                SELECT 
                              hm
                            , date
                            , avg(sent_message) as sent_message
                FROM

                    (SELECT
                          today()-1 as date
                        , formatDateTime(toStartOfFifteenMinutes(time), '%R') as hm
                        , count(user_id) as sent_message

                    FROM 
                       {db}.message_actions
                    WHERE 
                        time >=  today()-7 and time < today()

                    GROUP BY date, hm
                    ORDER BY date, hm)  

                GROUP BY date, hm
                ORDER BY date, hm      
                    '''
        df_cube_sent_message = ph.read_clickhouse(query, connection=connection)
        return df_cube_sent_message



    @task
    def extract_sent_message_today():
        query = ''' 
                SELECT 
                          toStartOfFifteenMinutes(time) as ts
                        , toDate(time) as date
                        , formatDateTime(ts, '%R') as hm
                        , count(user_id) as sent_message

                    FROM 
                        {db}.message_actions
                    WHERE 
                        time >= today() and time < toStartOfFifteenMinutes(now())

                    GROUP BY date, hm, ts
                    ORDER BY date, hm 
                    '''
        df_cube_sent_message_today = ph.read_clickhouse(query, connection=connection)
        return df_cube_sent_message_today


    #объединяю данные из 2 датафреймов в 1 для выявления аномалий в метриках
    @task
    def transform_data_feed(df_cube_feed_7day, df_cube_feed_today):

        df_cube_feed_7day['ts'] = df_cube_feed_7day['date'].astype ( str )+' '+df_cube_feed_7day['hm']
        df_cube_feed_7day['ts'] = pd.to_datetime(df_cube_feed_7day['ts'], format="%Y-%m-%d %H:%M")

        data_feed = pd.concat([df_cube_feed_7day, df_cube_feed_today], ignore_index=True)
        data_feed = data_feed.fillna(0)   # заменяю пропущенные значения NaN на 0  
        
        return data_feed

    @task
    def create_data_feed_dau(data_feed): 
        data_feed_dau = data_feed[['hm', 'date', 'ts', 'users_feed']].copy()
        return data_feed_dau
        
    @task
    def create_data_feed_views(data_feed): 
        data_feed_views = data_feed[['hm', 'date', 'ts', 'views']].copy()
        return data_feed_views

    @task
    def create_data_feed_likes(data_feed): 
        data_feed_likes = data_feed[['hm', 'date', 'ts', 'likes']].copy()
        return data_feed_likes

    @task
    def create_data_feed_ctr(data_feed): 
        data_feed_ctr =  data_feed[['hm', 'date', 'ts', 'ctr']].copy()
        return data_feed_ctr    


    #объединяю данные из 2 датафреймов в 1 для выявления аномалий в метриках
    @task
    def transform_data_sent_message(df_cube_sent_message, df_cube_sent_message_today):

        df_cube_sent_message['ts'] = df_cube_sent_message['date'].astype ( str )+' '+ df_cube_sent_message['hm']
        df_cube_sent_message['ts'] = pd.to_datetime(df_cube_sent_message['ts'], format="%Y-%m-%d %H:%M")

        data_sent_message = pd.concat([df_cube_sent_message, df_cube_sent_message_today], ignore_index=True)

        return data_sent_message


    @task
    def run_alerts(df_1, metrics):
        data = df_1
        metric = metrics


        # данные для подключения к боту
        my_token = 'token' # заменила настоящий токен, поскольку репозиторий публичный
        bot = telegram.Bot(token=my_token) # получаю доступ к бот
        chat_id = id_number # заменила настоящий id, поскольку репозиторий публичный

        
        # проверяем метрику на аномальность алгоритмом, описаным внутри функции check_anomaly()

        is_alert, current_value, diff = check_anomaly(data, metric, threshold=0.5) 
    
        if is_alert == 1:
            msg = '''Метрика {metric}:\nтекущее значение = {current_value:.2f}\nотклонение от вчера {diff:.2%}'''.format(metric=metric, current_value=current_value, diff=diff)

            sns.set(rc={'figure.figsize': (16, 10)}) # задаем размер графика
            plt.tight_layout()

            ax = sns.lineplot( # строим линейный график
                data = data.sort_values(by=['date', 'hm']), # задаем датафрейм для графика
                x="hm", y=metric, # указываем названия колонок в датафрейме для x и y
                hue="date" # задаем "группировку" на графике, т е хотим чтобы для каждого значения date была своя линия построена
                )

            for ind, label in enumerate(ax.get_xticklabels()): # этот цикл нужен чтобы разрядить подписи координат по оси Х,
                if ind % 5 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)
            ax.set(xlabel='time') # задаем имя оси Х
            ax.set(ylabel=metric) # задаем имя оси У

            ax.set_title('{}'.format(metric)) # задае заголовок графика
            ax.set(ylim=(0, None)) # задаем лимит для оси У

            # формируем файловый объект
            plot_object = io.BytesIO()
            ax.figure.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = '{0}.png'.format(metric)
            
            print(msg)
            plt.show()
            
            plt.close()

            # отправляем алерт
            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        return
        
    
    #вызов всех функций
    df_cube_feed_7day = extract_feed_7day()
    df_cube_feed_today = extract_feed_today()
    df_cube_users_message = extract_users_message()
    df_cube_sent_message = extract_sent_message()
    df_cube_sent_message_today = extract_sent_message_today()

    data_feed =  transform_data_feed(df_cube_feed_7day, df_cube_feed_today) 

    data_feed_dau = create_data_feed_dau(data_feed)
    data_feed_views = create_data_feed_views(data_feed)
    data_feed_likes = create_data_feed_likes(data_feed)
    data_feed_ctr = create_data_feed_ctr(data_feed)
    data_sent_message = transform_data_sent_message(df_cube_sent_message, df_cube_sent_message_today)

    run_alerts(df_1 = df_cube_users_message, metrics = 'users_message')
    run_alerts(df_1 = data_feed_dau, metrics = 'users_feed')
    run_alerts(df_1 = data_feed_views, metrics = 'views')
    run_alerts(df_1 = data_feed_likes, metrics = 'likes')
    run_alerts(df_1 = data_feed_ctr, metrics = 'ctr')
    run_alerts(df_1 = data_sent_message, metrics = 'sent_message')



aksakal_alert_less = aksakal_alert_less()
